## Misc functions for dealing with assembly data

Get subset of assemblies and use lineage info for a subset to get a filtered reference set. 

Then use as training data or for matching other strains in our assembly set to get their lineage.

In [1]:
import os, sys, io, random, subprocess, re
import string
import urllib
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width',200)
import pylab as plt

from IPython.display import Image
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

from mtbdiff import utils, analysis

## make bioproject table

may be useful later

In [26]:
from IPython.display import clear_output, display

def get_bioproject_table():
    proj = asm.BioProject.unique()
    print (len(proj))
    res=[]
    for id in proj:    
        clear_output(wait=True)
        if id.find('PRJNA|PRJEB'):
            id = id[5:]
        print (id)
        try:
            d = get_bioproject_info(id)
            s = pd.Series(d)
            res.append(s)
        except:
            pass

    res=pd.Data
    print (len(res))
    res.to_csv('../mtb_bioprojects.csv',index=False)

In [4]:
asm = utils.get_mtb_assembly_data()

In [28]:
bioproj = pd.read_csv('../mtbdiff/data/mtb_bioprojects.csv')

## get extra fields for mtb assemblies

In [ ]:

#asm['Assembly_nover'] = asm.Assembly.apply(lambda x: x.split('.')[0],1)
print (asm.columns)
variants = ['ofxr','haarlem','kzn','btb09','btb10','btb05','btb11']
species = ['bcg','bovis','h37rv','h37ra','cdc1551','beijing',           
           'africanum','africanum','marinum','canettii','ulcerans',
           'caprae','orygis','microti','pinnipedii']
def get_species(x):
    for s in species:
        if x.find(s) > 0:
            return s
    return 'mtb'

x=asm.Organism_Name.str.lower().apply(get_species)
x.value_counts()

In [49]:
#asm.Organism_Name.value_counts()[:20]
asm.BioProject.value_counts()[:10]

PRJNA343736    1643
PRJEB2138      1000
PRJEB5280       470
PRJEB2221       387
PRJNA476470     337
PRJEB2057       196
PRJEB2777        99
PRJNA407834      68
PRJEB2424        51
PRJNA288586      34
Name: BioProject, dtype: int64

In [ ]:
print (bioproj.columns)
bio=bioproj.fillna('-')
bioproj.Submitter_Organization.value_counts()[:10]
#x=bioproj[bioproj.Project_Description.str.contains('Beijing')]


## make filtered set

In [ ]:
icols = ['Assembly_nover','Strain','BioProject','#Organism/Name','species']
filtered = asm[(asm.Level=='Complete Genome') | (~asm.species.isin(['mtb']))]
print (len(filtered))
print (filtered.species.value_counts())

## select ref genomes for testing

In [5]:
d=[]
for i,g in asm.groupby('species'):
    d.append(g.iloc[0])    
#d = pd.DataFrame(d)
#d.to_csv('ref_genomes.csv',index=False)
refs = pd.read_csv('../mtbdiff/data/ref_genomes.csv')
